## ダミー変数、欠損を平均で補完、決定木

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

train = pd.read_csv("data/train.tsv",delimiter="\t")



### ageの欠損を平均で補完。embarkedは最も多いSで補完する

In [2]:
train['age']=train['age'].fillna(train['age'].mean())
train['embarked']=train['embarked'].fillna('S')
train.isnull().sum()

id          0
survived    0
pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    0
dtype: int64

### sexとembarkdedをダミー変数にする

In [3]:
train.head(2)

,id,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,3,1,1,female,35.0,1,0,53.10,S
1,4,0,3,male,35.0,0,0,8.05,S


In [4]:
x_train=pd.concat([train[['pclass','age','sibsp','parch','fare']],pd.get_dummies(train['sex']),pd.get_dummies(train['embarked'])],axis=1)

In [5]:
x_train.head()

,pclass,age,sibsp,parch,fare,female,male,C,Q,S
0,1,35.0,1,0,53.1000,1,0,0,0,1
1,3,35.0,0,0,8.0500,0,1,0,0,1
2,3,2.0,3,1,21.0750,0,1,0,0,1
3,2,14.0,1,0,30.0708,1,0,1,0,0
4,1,58.0,0,0,26.5500,1,0,0,0,1


### 目的変数を作る

In [6]:
y_train = train['survived']

### 訓練用と検証用にデータを分ける

In [95]:
from sklearn.model_selection import train_test_split
x_train_train, x_train_valid, y_train_train, y_train_valid = cross_validation.train_test_split(x_train, y_train, test_size=0.3)

### モデルを作ってみる

In [87]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier(random_state=0,max_depth=4)
model.fit(x_train_train, y_train_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [88]:
y_train_predicted=model.predict(x_train_train)

##### 教師データで確認すると９８％の正答率→ロジスティック回帰の時は80％ほどだった

In [89]:
(y_train_predicted==y_train_train).sum()/len(y_train_train)

0.86173633440514474

In [90]:
y_valid = model.predict(x_train_valid)
(y_valid==y_train_valid).sum()/len(y_train_valid)

0.79850746268656714

In [74]:
len(y_train_valid)

134

### 結局、分割の比率を7:3、max_depthを4にしたとき、検証用データの正解率が最大の0.7985となった。  
### これで投稿してみることにする。

## テスト用データを作る

In [91]:
test = pd.read_csv("data/test.tsv",delimiter="\t")
test_id = test['id']
test['age']=test['age'].fillna(test['age'].mean())
test['embarked']=test['embarked'].fillna('S')
test.isnull().sum()
x_test=pd.concat([test[['pclass','age','sibsp','parch','fare']],pd.get_dummies(test['sex']),pd.get_dummies(test['embarked'])],axis=1)

In [92]:
y_test = model.predict(x_test)
result = pd.concat([test_id,pd.DataFrame( y_test)], axis=1)

In [93]:
result.describe()

,id,0
count,446.000000,446.000000
mean,434.022422,0.313901
std,257.820954,0.464598
min,0.000000,0.000000
25%,215.750000,0.000000
50%,424.500000,0.000000
75%,653.750000,1.000000
max,890.000000,1.000000


In [94]:
result.to_csv('submit7.tsv',sep='\t',header=False,index=False)

## Signateのスコアは今まで一番よく0.779  
## でも、一番最初にロジスティック回帰を使ったときと大して差がない